In [6]:
# coding: utf-8
import import_ipynb

import Get_filelist as gf
import Read_pdf as rp
import Write_pdf as wp
import template as tp
import Write_word as ww
import Auto_Paste as ap

import textwrap
import os
import clipboard
import tkinter as tk
import pdfplumber
import configparser
import yaml
import PySimpleGUI as sg #新增
from shutil import copyfile
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import Menu
from tkinter import filedialog
from tkinter import simpledialog


class App:
    def __init__(self, master):
    #==============parameter set===============
        self.config = configparser.ConfigParser()
        self.config.read('config.ini')
        self.userdir = self.config['setting']['userpath'] #treeview 開啟資料夾用
        self.path = '' #dropdown menu開啟或列印檔案用

        try:
            template_yaml = self.config['setting']['template_yaml']
            with open(template_yaml, 'r', encoding="utf8") as yaml_file: #讀取template_yaml
                self.template_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
            print('讀取套版庫:' + template_yaml)
        except FileNotFoundError:
            sg.popup_ok('套版庫路徑錯誤，開始重新設定',font ='JhengHei 10')
            template_yaml = sg_select_yaml()
            with open(template_yaml, 'r', encoding="utf8") as yaml_file: #讀取template_yaml
                self.template_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
            print('讀取套版庫:' + template_yaml)
            modify_config(template_yaml)
            print('將套版庫路徑寫入設定檔:' + template_yaml)

        self.popup_yaml = os.path.join(os.getcwd(), self.config['setting']['popup_yaml'])
        with open(self.popup_yaml, 'r', encoding="utf8") as yaml_file: #讀取popup_yaml
            popup_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
        print('讀取字詞庫:' + self.popup_yaml)
            
        self.user_lexicon_yaml = os.path.join(os.getcwd(), self.config['setting']['user_lexicon_yaml'])
        with open(self.user_lexicon_yaml, 'r', encoding="utf8") as yaml_file: #讀取user_lexicon_yaml
            user_lexicon_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
        print('讀取差異原因字詞庫:' + self.user_lexicon_yaml)
        
        self.word_path = self.config['write_word']['word_path']
        print('寫入word的路徑:' + self.word_path)
        
        self.comp_code_path = self.config['setting']['comp_code_yaml']
        print('讀取公司別詞庫的路徑:' + self.comp_code_path)
        
        self.hotkey_set = ap.Hotkey_paste(
            self.config['auto_paste']['hotkey'], 
            float(self.config['auto_paste']['sleep_time'])
               )
        print('自動貼上熱鍵:{a}，間隔時間:{b}秒'.format(
            a = self.config['auto_paste']['hotkey'],
            b = self.config['auto_paste']['sleep_time'])
             )
        
    #==============GUI set====================
        #----root and frame set
        
        self.master = master
        self.master.title('自動決包說明輔助工具')
#         self.master.geometry('760x210')
        
        self.lf = ttk.Frame(self.master)
        self.rf1 = ttk.Frame(self.master)
        self.rf2 = ttk.Frame(self.master)
        self.bf = ttk.Frame(self.master)
        
        self.lf.grid(column = 0 , row = 0, rowspan = 2, padx = 5, sticky = 'NWSE')
        self.rf1.grid(column = 1 , row = 0, padx = 5, sticky = 'NWSE')
        self.rf2.grid(column = 1 , row = 1, padx = 5, pady = 8, sticky = 'NWSE')
        self.bf.grid(column = 0 , row = 2, columnspan = 2 , sticky = 'NWSE')
    
    #=================Bottom frame widget=============================
        #----Status Label
        self.statusbar = tk.Label(self.bf, text = '狀態列', bd = 1, relief= tk.SUNKEN, anchor = tk.W)
        self.statusbar.pack(fill = tk.X)
    
    #==============left frame widget set=================
        #---Label
        self.lbl_pdfList = tk.Label(self.lf, text = 'PDF檔案:')
        self.lbl_pdfList.pack(side = tk.TOP, anchor = tk.NW)
        #----Treeview
        head = ('案件名稱','路徑')
        self.tree = ttk.Treeview(self.lf, show = 'tree', height = 6)
        self.tree["columns"]=head
        self.tree.column("#0", width=200, minwidth=20)
        self.tree.column(head[0], width=250, minwidth=20)
        self.tree.column(head[1], width=0, minwidth=0)
        
        self.tree.bind('<ButtonRelease-1>', self.selectItem)
        self.tree.tag_configure('folder', background= "old lace")
        self.tree.pack()
        
        #----Button
        self.btn_refresh = ttk.Button(self.lf, text = '重新整理', command = self.tree_refresh)
        self.btn_refresh.pack(anchor = tk.SE, pady = 8)
        
        
        
    #==============Right frame1 widget set=================
        #----Label
        self.lbl_text = tk.Label(self.rf1, text = '議價說明:')
        self.lbl_text.grid(row = 0, column = 0 , sticky = tk.W)
        
        #---OptionMenu
        self.optmode_var = tk.StringVar(self.rf1)
#         self.optmode_var.set('簡易版')
        self.optmode = ttk.OptionMenu(self.rf1, self.optmode_var, *('簡易版', '簡易版', '三段式'), \
                                     command = self.optmode_SelectionEvent)
        self.optmode.grid(row = 0, column = 1, sticky = tk.E)
        
        self.opt_var = tk.StringVar(self.rf1)
        
        self.opt = ttk.OptionMenu(self.rf1, self.opt_var,\
                                  command = self.opt_SelectionEvent)
        self.opt.grid(row = 0 , column = 2 , sticky = tk.E)
        
        #----scrolledtext widget議價說明 &popup menu with right click
        self.txt = tk.scrolledtext.ScrolledText(self.rf1, height = 9, width = 76)
        self.popup_menu = Popup_menu(self.master, self.txt, self.statusbar)#scrolledtext傳入
        
        for topic in popup_data:
            self.popup_menu.add_cascade(topic, popup_data[topic])
        self.popup_menu.popup.add_separator()
        
        for topic in user_lexicon_data:
            self.popup_menu.add_cascade(topic, user_lexicon_data[topic])
        
        self.txt.grid(column = 0 , columnspan = 3 , row = 1, sticky = 'nse')
        self.txt.bind("<Button-3>", self.popup_menu.do_popup)#滑鼠右鍵啟動popupmenu
        
    #==============Right frame2 widget set=================
        #----Buttons
        self.btn_printPdf = ttk.Button(self.rf2, text = '列印PDF', command = self.printPDF)
        self.btn_printPdf.pack(side = tk.RIGHT)
        
        self.btn_openPdf = ttk.Button(self.rf2, text = '打開PDF', command = self.openPDF)
        self.btn_openPdf.pack(side = tk.RIGHT, padx = 5)
        
        self.btn_writePdf = ttk.Button(self.rf2, text = '寫入PDF', command = self.writePDF)
        self.btn_writePdf.pack(side = tk.RIGHT, padx = 5)
        
        self.btn_writePdf = ttk.Button(self.rf2, text = '寫入Word', command = self.writeWord)
        self.btn_writePdf.pack(side = tk.RIGHT, padx = 5)
        
        self.btn_bidinfo = ttk.Button(self.rf2, text = '參數列表', command = self.info_window)
        self.btn_bidinfo.pack(side = tk.RIGHT, padx = 5)
        
     #=================menu bar=============================
        self.menubar = tk.Menu(self.master)
        self.menu_file = tk.Menu(self.menubar, tearoff = 0)
        
        self.menu_file.add_command(label = '開啟資料夾', command = self.opendir)
        self.menu_file.add_separator()
        self.menu_file.add_command(label = '結束', command = self.on_closing)
        self.menubar.add_cascade(label='PDF檔案', menu = self.menu_file)
        
        self.menubar.add_command(label = '自動換行', command = self.wraptext)

        self.menu_edit_lexicon = tk.Menu(self.menubar, tearoff = 0)
        self.menu_edit_lexicon.add_command(label = '常用詞庫', command = lambda: 
                                           self.open_notepad(self.popup_yaml.replace('\\', '/')))
        self.menu_edit_lexicon.add_command(label = '差異詞庫', command = lambda: 
                                           self.open_notepad(self.user_lexicon_yaml.replace('\\', '/')))
        self.menu_edit_lexicon.add_command(label = '公司別對照詞庫', command = lambda: 
                                           self.open_notepad(self.comp_code_path.replace('\\', '/')))
        
        self.menubar.add_cascade(label='編輯詞庫', menu = self.menu_edit_lexicon)
        
        
        self.menu_select_setting_files = tk.Menu(self.menubar, tearoff = 0)
        self.menu_select_setting_files.add_command(label = '套版檔路徑', command = self.select_template_file)
        self.menu_select_setting_files.add_command(label = 'Word檔路徑', command = self.select_word_file)
        self.menu_select_setting_files.add_command(label = '公司別詞庫檔路徑', command = self.select_comp_code_file)
        self.menubar.add_cascade(label='設定載入檔路徑', menu = self.menu_select_setting_files)

        self.menubar.add_command(label = '導入自動貼上', command = self.to_erp)
        self.menubar.add_command(label = '關於', command = self.about)
        self.master.config(menu=self.menubar)
        self.tree_refresh()
        self.master.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def on_closing(self):
        self.hotkey_set.remove_hotkey()
        self.master.destroy()
    
    def to_erp(self):
        self.hotkey_set.text_to_erp = self.txt.get("1.0", tk.END)
        self.statusbar.config(text = '已導入決包說明，請至ERP說明表單中按下{}'.format(
            self.config['auto_paste']['hotkey']))
    
    def open_notepad(self, path):
        osCommandString = 'notepad.exe ' + path
        os.system(osCommandString)
    
    def opt_SelectionEvent(self, event):
        self.statusbar.config(text = '你選擇了' + self.opt_var.get() + ',可以按下開啟或列印選擇的檔案。')
    
    def optmode_SelectionEvent(self, event):
        self.statusbar.config(text = '你選擇了' + self.optmode_var.get())
        curItem = self.tree.focus()
        if self.tree.item(curItem)['values'][0] == 'None' or self.tree.item(curItem)['values'][0] == '案件名稱':
            return
        else:
            filename = self.tree.item(curItem)['text']
            self.pdf = os.path.join(self.tree.item(curItem)['values'][1], filename)
            if self.tree.item(curItem)['text'].split('.')[0][-2:] == '-w' :
                self.get_editText()
            else:
                self.make_submission()

            self.update_opt()
        
    def update_opt(self):
        # Reset var and delete all old options
#         if filename == None:
#             curItem = self.tree.focus()
#             filename = self.tree.item(curItem)['text']
        filename = os.path.basename(self.pdf)
        self.path = os.path.dirname(self.pdf)
        self.opt_var.set('')
        self.opt['menu'].delete(0, 'end')
        new_choices = []
        new_choices.append(filename)
#         full path = os.path.join(self.tree.item(curItem)['value'][1], self.rename(filename))        
        if os.path.isfile(self.rename(self.pdf)) :
            new_choices.append(self.rename(filename))
        if new_choices[0].split('.')[0][-2:] != '-w' and len(new_choices) == 2 :
            new_choices.reverse()
        self.opt_var.set(new_choices[0])
        # Insert list of new options (tk._setit hooks them up to var)
        for choice in new_choices:
            self.opt['menu'].add_command(label=choice, command=tk._setit(self.opt_var, choice))

    def tree_refresh(self, path= None):
        if path == None:
            path = self.userdir
        print('讀取資料夾:' + path)
        self.tree.delete(*self.tree.get_children())
        self.folder1=self.tree.insert("", 0, 'folder_erp',text="ERP開啟", values=self.tree['columns'],\
                                      open = True, tags = ('folder',))
        self.folder2=self.tree.insert("", 1, 'folder_original',text=path, values=self.tree['columns'],\
                                      open = True, tags = ('folder',))
        self.folder3=self.tree.insert("", 2, 'folder_edited',text=path+'(-w)', values=self.tree['columns'],\
                                      open = True, tags = ('folder',))       
        
        r = gf.walkpath(include_chars= self.config['get_filelist']['erp_file_chars'])
        r = gf.append_pdftext(r, title = 'ERP暫存區')
        self.insert_tree(self.folder1, r)
        
        r = gf.getFileList(path, filetype = '.pdf')
        r = gf.append_pdftext(r, title = path)

        original = []
        edited = []
        for item in r:
            if item[0].split('.')[0][-2:] == '-w':
#                 print(item)
                edited.append(item)
            else:
                original.append(item)
        self.insert_tree(self.folder2, original)
        self.insert_tree(self.folder3, edited)
        self.statusbar.config(text = '重新整理完畢。')
    
    def tree_refresh_w(self):#只更新treeview -w區域
        path = self.userdir
        self.tree.delete('folder_edited')
        r = gf.getFileList(self.userdir, filetype = '.pdf', include_chars = '-w')
        r = gf.append_pdftext(r, '-w檔案')

        self.folder3=self.tree.insert("", 2, 'folder_edited',text=path+'(-w)', values=self.tree['columns'],\
                                      open = True, tags = ('folder',))       
        self.insert_tree(self.folder3 , r)
        self.statusbar.config(text = '-w 資料區檔案重新整理完畢')
        
    def insert_tree(self, parent, li):
        for item in li:
            self.tree.insert(parent, "end",  text=item[0], iid = item[0], values=(item[2], item[1]))
    
    def make_submission(self):
        self.bid = rp.open(self.pdf)
        self.popup_menu.bid = self.bid
        
        #刪除scrolledtxt內容
        self.txt.delete('1.0', tk.END)
        if self.optmode_var.get() == '簡易版':
            self.txt.insert(tk.INSERT,tp.use_template(self.bid, self.template_data, easymode = True))
        else:
            self.txt.insert(tk.INSERT,tp.use_template(self.bid, self.template_data))

        
        self.statusbar.config(text='產生議價說明:'\
                              + self.bid.id + '-' + self.bid.comp+ '-' + self.bid.sn + '-' + self.bid.name)
    
    def get_editText(self): #讀取已經寫入過的pdf
        self.bid = rp.open(self.pdf)
        pos = {'x0': 50.000, 'top': 385.391, 'x1':818.000, 'bottom': 498.191}
        pdf_1 = pdfplumber.open(self.pdf)
        p0 = pdf_1.pages[0]
        a = rp.get_text(p0,pos)
        self.txt.delete('1.0', tk.END)
        self.txt.insert(tk.INSERT, a)
        self.statusbar.config(text='讀取'+ self.pdf + '的文字完成。')
    
    def selectItem(self, event):#treeview 選擇時的事件
        curItem = self.tree.focus()
#         print(self.tree.item(curItem))
        
        if self.tree.item(curItem)['values'][0] == 'None' or self.tree.item(curItem)['values'][0] == '案件名稱':
            self.statusbar.config(text='你選取的不是可載入的檔案。')
        else:
            filename = self.tree.item(curItem)['text']
            self.pdf = os.path.join(self.tree.item(curItem)['values'][1], filename)
            if self.tree.item(curItem)['text'].split('.')[0][-2:] == '-w' :
                self.get_editText()
            else:
                self.make_submission()

            self.update_opt()
            
    def select_comp_code_file(self):
        path = sg.popup_get_file('請選擇要預設要讀取的公司別對照yaml檔案:\n目前的設定檔為'+ self.comp_code_path ,
                                    file_types = (('YAML', '*.yaml'),),
                                    initial_folder = os.getcwd(),
                                    title = '公司別對照yaml檔設定',font ='JhengHei 10')
                
        if path != None:
            self.comp_code_path = path
            modify_config(self.comp_code_path, section = 'setting', key = 'comp_code_yaml')
            self.statusbar.config(text = '將公司別對照yaml路徑寫入設定檔:' + path)
        else:
            self.statusbar.config(text = '取消公司別對照yaml路徑設定')
    def select_word_file(self):
        path = sg.popup_get_file('請選擇要預設要寫入的決包說明word檔案(只能是docx檔):\n目前的設定檔為'+ self.word_path ,
                                    file_types = (('Docx', '*.docx'),),
                                    initial_folder = os.getcwd(),
                                    title = '決包說明word檔設定',font ='JhengHei 10')
                
        if path != None:
            self.word_path = path
            modify_config(self.word_path, section = 'write_word', key = 'word_path')
            self.statusbar.config(text = '將word檔路徑寫入設定檔:' + self.word_path)
        else:
            self.statusbar.config(text = '取消word檔路徑設定')
    def select_template_file(self):
        
        while True:
            template_file_path = sg.popup_get_file('請選擇要預設要載入的套版檔案(ex:template.yaml):\n目前的設定檔為' +self.config['setting']['template_yaml'] ,
                                               default_path = os.getcwd() +'\\template_cn.yaml',
                                               file_types = (('YAML', '*.yaml'),),
                                               initial_folder = os.getcwd(),
                                               title = '套版檔設定', font ='JhengHei 10')
            if template_file_path == None:
                break
            elif os.path.isfile(template_file_path) and os.path.splitext(template_file_path)[-1] == '.yaml':
                break
            else:
                sg.popup_ok('檔案選擇錯誤，請選擇正確的template.yaml檔', font = 'jengHei 10')
                
        
        if template_file_path != None:
            modify_config(template_file_path)
            self.config = configparser.ConfigParser()
            self.config.read('config.ini')
            with open(template_file_path, 'r', encoding="utf8") as yaml_file: #讀取template_yaml
                self.template_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
            
            self.statusbar.config(text='套版檔案設定完成:{}'.format(template_file_path))
        else:
            self.statusbar.config(text='取消套版檔案設定')
        
    def wraptext(self):
        ans = simpledialog.askinteger('自動決包說明輔助工具', '自動換行的行數', initialvalue=38)

        if ans != None:
            text = self.txt.get("1.0", tk.END)
            self.txt.delete('1.0', tk.END)
            text = autowrap(text, ans)
            self.txt.insert(tk.INSERT, text)
        
    def info_window(self):
        win_info = tk.Toplevel(self.master)
        infoApp = info_app(win_info, self.bid)
    
    def remove(self):
        curItem = self.tree.focus()
        if self.tree.item(curItem)['values'][0] == 'None' or self.tree.item(curItem)['values'][0] == '案件名稱':
            self.statusbar.config(text='你選取的不是檔案。')
        else:
            full_path = os.path.join(self.tree.item(curItem)['values'][1],\
                                     self.tree.item(curItem)['text'])
            del self.bid
            os.remove(full_path)
            self.statusbar.config(text ='刪除'+ full_path +'完成。')
        
        self.opt_var.set('')
        self.opt['menu'].delete(0, 'end')
        
    def writeWord(self):
        if self.word_path == '':
            self.select_word_file()
        else:
            try:
                title = '-'.join([self.bid.id, self.bid.comp, self.bid.sn, self.bid.name])
                text = self.txt.get("1.0", tk.END)
                ww.write_word(self.word_path, title, text)
                self.statusbar.config(text='將文字寫入{}完成'.format(self.word_path))
            except AttributeError:
                sg.popup_ok('還沒有選擇要產生說明的PDF檔',font ='JhengHei 10')
#             except :
#                 sg.popup_ok('Word檔設定錯誤，請重新設定',font ='JhengHei 10')
                
    def writePDF(self):
        userpath = self.tree.item(self.folder2)['text']
        if 'INetCache' in self.pdf:#判斷要寫入的檔案來源是否來自ERP的暫存資料
            sep = '-'
            filename = sep.join((self.bid.id, self.bid.comp, self.bid.sn)) + '.pdf'
            full_path = os.path.join(userpath, filename)
            copyfile(self.pdf, full_path)
            self.statusbar.config(text = '複製ERP開啟檔案到' + full_path)
            self.pdf = full_path
        elif os.path.basename(self.pdf).split('.')[0][-2:] == '-w':
            if os.path.isfile(self.rename(self.pdf)) :
                self.pdf =self.rename(self.pdf)
            else:
                self.statusbar.config(text = '沒有原始檔案供寫入，請至ERP再下載一次。')
                return
            
        myText = self.txt.get("1.0", tk.END)
        self.statusbar.config(text='將文字寫入'+ self.rename(self.pdf) + '...')
        wp.write_pdf(self.pdf, myText, 12)
        self.statusbar.config(text='將文字寫入'+ self.rename(self.pdf) + '完成。')
        self.tree_refresh_w
        self.update_opt()
    
    def printPDF(self):
        filename = self.opt_var.get()
        full_path = os.path.join(self.path, filename)
        self.statusbar.config(text='列印'+ full_path + '...')
        os.startfile(full_path,'print')
    
    def openPDF(self):
        filename = self.opt_var.get()
        full_path = os.path.join(self.path, filename)
        self.statusbar.config(text='打開'+ full_path + '...')
        os.startfile(full_path)
        
    def about(self):
        text = '''
            Something bigger then yourself to make you keep walking...
        '''
        tk.messagebox.showinfo('自動決包說明輔助工具', text)
    
    def opendir(self):
        self.userdir = tk.filedialog.askdirectory(initialdir = 'd:/')
        
        if self.userdir != '':
            config = configparser.ConfigParser()
            config.read('config.ini')
            config['setting']['userpath']= self.userdir
            with open('config.ini', 'w') as configfile:
                config.write(configfile)

            self.tree_refresh(self.userdir)
            self.statusbar.config(text='讀取'+ self.userdir + '內的PDF檔完成')
        
    
    #寫入的pdf檔重新命名並製作新的路徑
    def rename(self, path):
        nf = os.path.basename(path).split('.')
        if nf[0][-2:] == '-w':
            f = path.replace('-w','')
        else:
            f = path.replace(nf[0], nf[0] + '-w')
        return f

def autowrap(text, width):  
    text = text.split('\n')
    #偵測項目符號字元，自動計算縮排空格數
    a = text[0][0:2]#取出第一行前2碼
    indent_char = ''
    for i in range(len(a.encode('gbk'))):
        indent_char = indent_char+' '
    
    newtext = ''
    for element in text:
        if len(element) < width:
            newtext = newtext + element + '\n'
        else:
            wrapt = textwrap.wrap(element, width)
            for x in range(len(wrapt)):
                if x == 0:
                    newtext = newtext + wrapt[x] + '\n'
                else:
                    newtext = newtext + textwrap.indent(wrapt[x], indent_char) + '\n'

    return newtext

def sg_select_yaml():
    while True:
                template_file_path = sg.popup_get_file('請選擇要預設要載入的套版檔案(ex:template.yaml):' ,
                                                   default_path = os.getcwd() +'\\template_cn.yaml',
                                                   file_types = (('YAML', '*.yaml'),),
                                                   initial_folder = os.getcwd(),
                                                   title = '套版檔設定',font ='JhengHei 10')

                if template_file_path == None:
                    sg.popup_ok('沒有選擇檔案，無法執行程式，請選擇正確的套版檔', font = 'jengHei 10')

                elif os.path.isfile(template_file_path) and os.path.splitext(template_file_path)[-1] == '.yaml':
                    with open(template_file_path, 'r', encoding = 'utf8') as stream:
                        yaml_data = yaml.load(stream, Loader = yaml.FullLoader)

                    try: 
                        if yaml_data['yaml_type'] == 'template':    
                            return template_file_path
                            break
                    except KeyError:
                        sg.popup_ok('非套版檔案(ex:template.yaml)，請選擇正確的檔案', font = 'jengHei 10')
                else:
                    sg.popup_ok('檔案選擇錯誤，請選擇正確的套版檔', font = 'jengHei 10')


def modify_config(value, **option):
    option.setdefault('filename', 'config.ini')
    option.setdefault('section', 'setting')
    option.setdefault('key', 'template_yaml')
    
    config_file_path = os.path.join(os.getcwd(), option['filename'])
    config = configparser.ConfigParser()
    config.optionxform = str  #reference: http://docs.python.org/library/configparser.html
    config.read(config_file_path)

    config.set(option['section'], option['key'], value)#要修改config.ini Key 的 Value
    with open(config_file_path, 'w') as configfile:
                config.write(configfile)
                    

class info_app:
    def __init__(self, master, bid):
        self.master = master
        self.master.title('案件資訊')
        self.topframe = tk.LabelFrame(self.master, width= 700, text = '案件資訊')
        self.bottomframe = tk.LabelFrame(self.master, width = 700, text = '報價資訊')
        self.submit_btn = ttk.Button(self.master, text='關閉', command=self.submit)
        self.previous_btn = ttk.Button(self.master, text = '上一筆', command = self.previous_vnd)
        self.next_btn = ttk.Button(self.master, text = '下一筆', command = self.next_vnd)
        
        self.topframe.pack(padx = 10, pady = 5)
        self.bottomframe.pack(padx =10 , pady = 5)
        self.previous_btn.pack(padx = 5 ,pady = 5, anchor = tk.W, side = tk.LEFT)
        self.submit_btn.pack(pady = 5, anchor = tk.CENTER, side = tk.LEFT)
        self.next_btn.pack(padx = 5, pady = 5 , anchor = tk.E, side = tk.LEFT)
        self.sn = 0
        self.bid = bid


        self.labelsBid = ['工程編號','公司','次別','專業類別','等級','工程名稱', '幣別', '性質', '預算']
        self.labelsQuo = ['報價次序', '廠商名稱','廠商編號','報價方式','廠商等級','報價金額','報價稅率', 
                          '議後金額', '議價稅率','議價次數']
        

        for i in range(len(self.labelsBid)):
            self.cur_label = ttk.Label(self.topframe, text = self.labelsBid[i])
            self.cur_label.grid(row=i, column=0, sticky=tk.NW, pady = 2)

        for i in range(len(self.labelsQuo)):
            self.cur_label = ttk.Label(self.bottomframe, text = self.labelsQuo[i])
            self.cur_label.grid(row = i , column = 0 , sticky = tk.W)

        self.name_var = tk.StringVar() 
        self.bid_info = {
            'id': tk.StringVar(),
            'comp': tk.StringVar(),
            'sn': tk.StringVar(),
            'category': tk.StringVar(),
            'level': tk.StringVar(),
            'name': tk.StringVar(),
            'currency': tk.StringVar(),
            'type': tk.StringVar(),
            'budget' : tk.StringVar(),
        }

        self.quo_info = {
            'vndSn' : tk.StringVar(),
            'vndName' : tk.StringVar(),
            'vndID' : tk.StringVar(),
            'vndType': tk.StringVar(),
            'vndLevel': tk.StringVar(),
            'vndQuo': tk.StringVar(),
            'vndQuoRate': tk.StringVar(),
            'vndFinalprice': tk.StringVar(),
            'vndFinalRate': tk.StringVar(),
            'vndBargainTimes': tk.StringVar()
        }


        counter = 0
        for i in self.bid_info:
            self.cur_entrybox = ttk.Entry(self.topframe, width=28 , textvariable=self.bid_info[i])
            self.cur_entrybox.grid(row=counter, column= 1, sticky = tk.E, padx = 5)
            counter += 1 

        counter = 0
        for i in self.quo_info:
            self.cur_entrybox = ttk.Entry(self.bottomframe, width=28 , textvariable=self.quo_info[i])
            self.cur_entrybox.grid(row=counter, column= 1, sticky = tk.E, padx = 5)
            counter += 1 
        
        self.initial_bid(self.bid)
        self.initial_quo(self.bid)

    def initial_bid(self, bid):
        self.bid_info['name'].set(bid.name)
        self.bid_info['id'].set(bid.id)
        self.bid_info['comp'].set(bid.comp)
        self.bid_info['sn'].set(bid.sn)
        self.bid_info['category'].set(bid.category)
        self.bid_info['level'].set(bid.level)
        self.bid_info['currency'].set(bid.currency)
        self.bid_info['type'].set(bid.type)
        self.bid_info['budget'].set(bid.budget)
    
    def initial_quo(self, bid):
        self.quo_info['vndSn'].set(bid.vnd[self.sn]['sn'])
        self.quo_info['vndName'].set(bid.vnd[self.sn]['name'])
        self.quo_info['vndID'].set(bid.vnd[self.sn]['id'])
        self.quo_info['vndType'].set(bid.vnd[self.sn]['type'])
        self.quo_info['vndLevel'].set(bid.vnd[self.sn]['level'])
        self.quo_info['vndQuo'].set(bid.vnd[self.sn]['quo'])
        self.quo_info['vndQuoRate'].set(bid.vnd[self.sn]['quoRate'])
        self.quo_info['vndFinalprice'].set(bid.vnd[self.sn]['finalPrice'])
        self.quo_info['vndFinalRate'].set(bid.vnd[self.sn]['finalRate'])
        self.quo_info['vndBargainTimes'].set(bid.vnd[self.sn]['bargain_times'])
    
    def submit(self):
        self.master.destroy()
    
    def previous_vnd(self):
        if self.sn != 0:
            self.sn += -1
            self.initial_quo(self.bid)
    
    def next_vnd(self):
        if self.sn != len(self.bid.vnd) - 1:
            self.sn += 1
            self.initial_quo(self.bid)

class Popup_menu(tk.Menu):

    def __init__(self, parent, st, statusbar):
        tk.Menu.__init__(self, parent,)
        # create a menu
        self.parent = parent
        self.scrolledtxt = st
        self.stbar = statusbar
        self.var = tk.StringVar()
        self.popup = tk.Menu(parent, tearoff=0)
        self.popup.add_command(label="複製", command = self.copy) # , command=next) etc...
        self.popup.add_command(label="剪下", command = self.cut)
        self.popup.add_command(label="貼上", command = self.paste)
        self.popup.add_command(label="自動換行", command = self.warp_text)
        self.popup.add_command(label="重新載入詞庫", command = self.refresh)
        self.popup.add_separator()
        self.popup.add_command(label='{}內字詞取代', command = self.format_text)
        self.popup.add_separator()
#         self.popup_vnd =tk.Menu(self.popup, tearoff = False)
        
    def refresh(self):
        self.popup.delete(0, tk.END)
        self.__init__(self.parent, self.scrolledtxt, self.stbar)

        config = configparser.ConfigParser()
        config.read('config.ini')
        
        popup_yaml = os.path.join(os.getcwd(), config['setting']['popup_yaml'])
        with open(popup_yaml, 'r', encoding="utf8") as yaml_file: #讀取popup_yaml
            popup_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
        print('讀取字詞庫:' + popup_yaml)
        
        user_lexicon_yaml = os.path.join(os.getcwd(), config['setting']['user_lexicon_yaml'])
        with open(user_lexicon_yaml, 'r', encoding="utf8") as yaml_file: #讀取user_lexicon_yaml
            user_lexicon_data = yaml.load(yaml_file, Loader = yaml.FullLoader )
        print('讀取差異原因字詞庫:' + user_lexicon_yaml)
        
        
        
        for topic in popup_data:
            self.add_cascade(topic, popup_data[topic])
        self.popup.add_separator()
        
        for topic in user_lexicon_data:
            self.add_cascade(topic, user_lexicon_data[topic])
            
        self.stbar.config(text = '字詞庫重新載入完畢。')
    

    
    def add_cascade(self, label, item_list):
        self.popup_lexicon = tk.Menu(self.popup, tearoff= False)
        self.popup.add_cascade(label= label , menu = self.popup_lexicon)
        for item in item_list:
            self.popup_lexicon.add_radiobutton(value=item, label=item, variable=self.var, command = self.callback)
        
    def warp_text(self):
        text = self.scrolledtxt.get("1.0", tk.END)
        self.scrolledtxt.delete('1.0', tk.END)
        text = autowrap(text, 38)
        self.scrolledtxt.insert(tk.INSERT, text)
    
    def format_text(self):
        text = self.scrolledtxt.get("1.0", tk.END)
        text = tp.format_text(text, self.bid)
        self.scrolledtxt.delete('1.0', tk.END)
        self.scrolledtxt.insert(tk.INSERT,text)
    
    def callback(self):
        print('scrolledtext popup_menu selected:', self.var.get())
        self.scrolledtxt.insert(tk.INSERT, self.var.get())
        
    def copy(self, event=None):
        self.scrolledtxt.clipboard_clear()
        text = self.scrolledtxt.get("sel.first", "sel.last")
        self.scrolledtxt.clipboard_append(text)

    def cut(self, event=None):
        self.copy()
        self.scrolledtxt.delete("sel.first", "sel.last")

    def paste(self, event=None):
        text = self.scrolledtxt.selection_get(selection='CLIPBOARD')
        self.scrolledtxt.insert('insert', text)

    def do_popup(self, event):
        # display the popup menu
        try:
            self.popup.tk_popup(event.x_root, event.y_root, 0)
        finally:
            # make sure to release the grab (Tk 8.0a1 only)
            self.popup.grab_release()

        
if __name__ == '__main__':
    
    sg.theme('LightGreen')
    filepath = os.path.join(os.getcwd(), 'config.ini')
    if not os.path.isfile(filepath):#沒有config.ini時
        config = configparser.ConfigParser()
        config.add_section('setting')
        config['setting']['userpath'] = 'd:/'
        config['setting']['popup_yaml'] = 'hot_phrases.yaml'
        sg.popup_ok('第一次使用，須先設定載入套版檔(template.yaml)',font ='JhengHei 10')
        config['setting']['template_yaml'] = sg_select_yaml()
        config['setting']['comp_code_yaml'] = 'comp_code.yaml'
        config['setting']['user_lexicon_yaml']= 'rformat.yaml'
        config['setting']['warp_chars'] = '38'
        config.add_section('get_filelist')
        config['get_filelist']['erp_file_chars'] = 'getjobid'
        config.add_section('write_word')
        config['write_word']['word_path'] = ''
        config.add_section('auto_paste')
        config['auto_paste']['hotkey'] = 'ctrl+3'
        config['auto_paste']['sleep_time'] = '0.5'

        with open(filepath, 'w') as configfile:
            config.write(configfile)
        print('沒有偵測到設定檔，重新建立'+ filepath)
    else:
        print('偵測到設定檔:'+ filepath)
    
    root = tk.Tk()
    app = App(root)
    root.mainloop()
    
    

沒有偵測到設定檔，重新建立C:\Users\spp43\bid_auto_submission\config.ini
讀取套版庫:C:\Users\spp43\bid_auto_submission\template_cn.yaml
讀取字詞庫:C:\Users\spp43\bid_auto_submission\hot_phrases.yaml
讀取差異原因字詞庫:C:\Users\spp43\bid_auto_submission\rformat.yaml
寫入word的路徑:
讀取公司別詞庫的路徑:comp_code.yaml
自動貼上熱鍵:ctrl+3，間隔時間:0.5秒
讀取資料夾:d:/
讀取資料夾:D:/autosub_source
套版選擇:[一般案][有議價(超預算)]
套版選擇:[一般案][無議價]
套版選擇:[一般案][有議價(超預算)]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[預約子案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[變更案][簡易版]
套版選擇:[一般案][有議價(超預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(超預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
套版選擇:[一般案][有議價(低預算)]
熱鍵移除完畢
